# Test Face_congnition [Extract face from corresponding image]
* 원하는 이미지에서 얼굴만 추출하는지 확인할 수 있는 코드

In [3]:
from PIL import Image
import face_recognition

# Load the jpg file into a numpy array
image = face_recognition.load_image_file("two_people.jpg")

# Find all the faces in the image using a pre-trained convolutional neural network.
# This method is more accurate than the default HOG model, but it's slower
# unless you have an nvidia GPU and dlib compiled with CUDA extensions. But if you do,
# this will use GPU acceleration and perform well.
# See also: find_faces_in_picture.py
face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="cnn")

print("I found {} face(s) in this photograph.".format(len(face_locations)))

for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

I found 2 face(s) in this photograph.
A face is located at pixel location Top: 34, Left: 139, Bottom: 129, Right: 234
A face is located at pixel location Top: 34, Left: 465, Bottom: 129, Right: 560


# Test Face_recognition [ compare image]
* 원하는 이미지를 데이터로 추가한후 아무 사진을 보여 주었을때 인식하는지 여부 파악하는 코드

In [2]:
import face_recognition
from PIL import Image, ImageDraw
import numpy as np

# Load a sample picture and learn how to recognize it.
obama_image = face_recognition.load_image_file("obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# Load a second sample picture and learn how to recognize it.
biden_image = face_recognition.load_image_file("biden.jpg")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]
# Load a sample picture and learn how to recognize it.
test01_image = face_recognition.load_image_file("test01.jpg")
test01_face_encoding = face_recognition.face_encodings(test01_image)[0]

# Load a sample picture and learn how to recognize it.
test02_image = face_recognition.load_image_file("test02.jpg")
test02_face_encoding = face_recognition.face_encodings(test02_image)[0]

# Load a sample picture and learn how to recognize it.
test03_image = face_recognition.load_image_file("test03.jpg")
test03_face_encoding = face_recognition.face_encodings(test03_image)[0]

# Load a sample picture and learn how to recognize it.6
test04_image = face_recognition.load_image_file("test04.jpg")
test04_face_encoding = face_recognition.face_encodings(test04_image)[0]

test05_image = face_recognition.load_image_file("test05.jpg")
test05_face_encoding = face_recognition.face_encodings(test05_image)[0]

test08_image = face_recognition.load_image_file("test08.jpg")
test08_face_locations = face_recognition.face_locations(test08_image)
test08_face_encoding = face_recognition.face_encodings(test08_image,test08_face_locations,num_jitters=100)[0]

# test09_image = face_recognition.load_image_file("test09.jpg")
# test09_face_encoding = face_recognition.face_encodings(test09_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    obama_face_encoding,
    biden_face_encoding,
    test01_face_encoding,
    test02_face_encoding,
    test03_face_encoding,
    test04_face_encoding,
    test05_face_encoding,
    test08_face_encoding,
]

known_face_names = [
    "Barack Obama",
    "Joe Biden",
    "test01",
    "test02",
    "test03",
    "test04",
    "test05",
    "test08",
]

# Load an image with an unknown face
unknown_image = face_recognition.load_image_file("two_people.jpg")

# Find all the faces and face encodings in the unknown image
face_locations = face_recognition.face_locations(unknown_image,number_of_times_to_upsample=2,model="cnn")
face_encodings = face_recognition.face_encodings(unknown_image, face_locations,num_jitters=100)

# Convert the image to a PIL-format image so that we can draw on top of it with the Pillow library
# See http://pillow.readthedocs.io/ for more about PIL/Pillow
pil_image = Image.fromarray(unknown_image)
# Create a Pillow ImageDraw Draw instance to draw with
draw = ImageDraw.Draw(pil_image)

# Loop through each face found in the unknown image
for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    # See if the face is a match for the known face(s)
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding,tolerance=0.4)

    name = "Unknown"

    # If a match was found in known_face_encodings, just use the first one.
    # if True in matches:
    #     first_match_index = matches.index(True)
    #     name = known_face_names[first_match_index]

    # Or instead, use the known face with the smallest distance to the new face
    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
        name = known_face_names[best_match_index]

    # Draw a box around the face using the Pillow module
    draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

    # Draw a label with a name below the face
    text_width, text_height = draw.textsize(name)
    draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
    draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))


# Remove the drawing library from memory as per the Pillow docs
del draw

# Display the resulting image
pil_image.show()


MemoryError: bad allocation

# Code for adding percentages

In [3]:
import face_recognition

# Often instead of just checking if two faces match or not (True or False), it's helpful to see how similar they are.
# You can do that by using the face_distance function.

# The model was trained in a way that faces with a distance of 0.6 or less should be a match. But if you want to
# be more strict, you can look for a smaller face distance. For example, using a 0.55 cutoff would reduce false
# positive matches at the risk of more false negatives.

# Note: This isn't exactly the same as a "percent match". The scale isn't linear. But you can assume that images with a
# smaller distance are more similar to each other than ones with a larger distance.

# Load some images to compare against
known_obama_image = face_recognition.load_image_file("obama.jpg")
known_biden_image = face_recognition.load_image_file("biden.jpg")

# Get the face encodings for the known images
obama_face_encoding = face_recognition.face_encodings(known_obama_image)[0]
biden_face_encoding = face_recognition.face_encodings(known_biden_image)[0]

known_encodings = [
    obama_face_encoding,
    biden_face_encoding
]

# Load a test image and get encondings for it
image_to_test = face_recognition.load_image_file("obama2.jpg")
image_to_test_encoding = face_recognition.face_encodings(image_to_test)[0]

# See how far apart the test image is from the known faces
face_distances = face_recognition.face_distance(known_encodings, image_to_test_encoding)

for i, face_distance in enumerate(face_distances):
    print("The test image has a distance of {:.2} from known image #{}".format(face_distance, i))
    print("- With a normal cutoff of 0.6, would the test image match the known image? {}".format(face_distance < 0.6))
    print("- With a very strict cutoff of 0.5, would the test image match the known image? {}".format(face_distance < 0.5))

The test image has a distance of 0.32 from known image #0
- With a normal cutoff of 0.6, would the test image match the known image? True
- With a very strict cutoff of 0.5, would the test image match the known image? True
The test image has a distance of 0.84 from known image #1
- With a normal cutoff of 0.6, would the test image match the known image? False
- With a very strict cutoff of 0.5, would the test image match the known image? False
